In [1]:
!pip install  optuna xgboost imbalanced-learn


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

import dagshub
dagshub.init(repo_owner='Vaibha3246', repo_name='influence_mirror', mlflow=True)

import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("https://dagshub.com/Vaibha3246/influence_mirror.mlflow")

Accessing as Vaibha3246

Initialized MLflow to track repo "Vaibha3246/influence_mirror"

Repository Vaibha3246/influence_mirror initialized!

In [3]:
# set or create an experiment
mlflow.set_experiment("exp_5 ml_algo_with_hp_tunning")


2025/10/11 10:44:38 INFO mlflow.tracking.fluent: Experiment with name 'exp_5 ml_algo_with_hp_tunning' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/adb82bab710d416190b0fea77cabca06', creation_time=1760159677050, experiment_id='3', last_update_time=1760159677050, lifecycle_stage='active', name='exp_5 ml_algo_with_hp_tunning', tags={}>

In [4]:
import optuna
import mlflow
import mlflow.xgboost
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import scipy.sparse as sp
import numpy as np
import pandas as pd

e:\influence-mirror\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [8]:
import seaborn as sns 

In [5]:

df=pd.read_csv('preprocessing.csv').dropna(subset=['text_clean'])

In [10]:
# -------------------------
# Step 1: Clean target
# -------------------------
df['sentiment_numeric'] = df['sentiment_numeric'].map({-1: 2, 0: 0, 1: 1})
df = df.dropna(subset=['sentiment_numeric'])

# -------------------------
# Step 2: Select features
# -------------------------
numeric_cols = [col for col in df.columns if col not in [
    'video_id', 'category', 'text', 'text_clean', 'sentiment',
    'dominant_emotion', 'published_at', 'sentiment_numeric'
]]

X_numeric = df[numeric_cols]
y = df['sentiment_numeric']

# Scale numeric features
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)

# Train/test split
X_train_num, X_test_num, y_train, y_test, train_idx, test_idx = train_test_split(
    X_numeric_scaled, y, df.index, test_size=0.2, random_state=42, stratify=y
)

# -------------------------
# Step 3: TF-IDF for text
# -------------------------
tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=10000)
X_train_text = tfidf.fit_transform(df.loc[train_idx, 'text_clean'])
X_test_text = tfidf.transform(df.loc[test_idx, 'text_clean'])

# Combine numeric + text
X_train = sp.hstack([X_train_text, sp.csr_matrix(X_train_num)])
X_test = sp.hstack([X_test_text, sp.csr_matrix(X_test_num)])

# -------------------------
# Step 4: Apply SMOTE
# -------------------------
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# -------------------------
# Step 5: Define Optuna objective
# -------------------------
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 400),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.5, 2.0),
        "objective": "multi:softmax",       # <-- returns class labels directly
        "num_class": len(np.unique(y)),
        "random_state": 42,
        "n_jobs": -1
    }

    model = XGBClassifier(**params)
    model.fit(X_train_res, y_train_res)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    return acc

# -------------------------
# Step 6: Run Optuna tuning
# -------------------------
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=25)

# -------------------------
# Step 7: Train best XGBoost model
# -------------------------
best_params = study.best_params
best_model = XGBClassifier(**best_params)
best_model.fit(X_train_res, y_train_res)
y_pred = best_model.predict(X_test)

# -------------------------
# Step 8: Log metrics + model in MLflow
# -------------------------
with mlflow.start_run(run_name="XGBoost_TFIDF_SMOTE_Optuna_Best"):
    # Log hyperparameters
    mlflow.log_params(best_params)
    mlflow.log_param("vectorizer_type", "TF-IDF")
    mlflow.log_param("ngram_range", (1,3))
    mlflow.log_param("max_features", 10000)
    mlflow.log_param("imbalance_method", "SMOTE")

    # Log accuracy
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))

    # Log full classification report metrics
    classification_rep = classification_report(y_test, y_pred, output_dict=True)
    for label, metrics in classification_rep.items():
        if isinstance(metrics, dict):
            for metric, value in metrics.items():
                mlflow.log_metric(f"{label}_{metric}", value)

    # Log confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix: XGBoost Best Model")
    plt.savefig("confusion_matrix_xgboost_best.png")
    mlflow.log_artifact("confusion_matrix_xgboost_best.png")
    plt.close()

    # Log trained model
    mlflow.xgboost.log_model(best_model, "xgboost_best_model")

print("🎯 MLflow run logged successfully for XGBoost ✅")

[I 2025-10-11 21:03:43,400] A new study created in memory with name: no-name-254d6a6f-1c96-46b5-ad0f-0bdddd44a393
[I 2025-10-11 21:05:12,075] Trial 0 finished with value: 0.7661758143685855 and parameters: {'n_estimators': 315, 'max_depth': 8, 'learning_rate': 0.03745891237904555, 'subsample': 0.869164239375805, 'colsample_bytree': 0.6906325659196921, 'gamma': 0.8795923404622225, 'min_child_weight': 10, 'reg_alpha': 0.08164439998607109, 'reg_lambda': 1.117189284477916}. Best is trial 0 with value: 0.7661758143685855.
[I 2025-10-11 21:05:45,067] Trial 1 finished with value: 0.7714190093708166 and parameters: {'n_estimators': 339, 'max_depth': 4, 'learning_rate': 0.1407850936758751, 'subsample': 0.9486371399409629, 'colsample_bytree': 0.8451784557246457, 'gamma': 2.64009369294226, 'min_child_weight': 8, 'reg_alpha': 0.9367292715293631, 'reg_lambda': 1.7362511250058374}. Best is trial 1 with value: 0.7714190093708166.
[I 2025-10-11 21:06:44,395] Trial 2 finished with value: 0.754239178937

🏃 View run XGBoost_TFIDF_SMOTE_Optuna_Best at: https://dagshub.com/Vaibha3246/influence_mirror.mlflow/#/experiments/3/runs/36d2471a2b944f819278cf08c3c07413
🧪 View experiment at: https://dagshub.com/Vaibha3246/influence_mirror.mlflow/#/experiments/3
🎯 MLflow run logged successfully for XGBoost ✅
